## Preprocessing

In [2]:
from openfe import OpenFE, tree_to_formula, transform, TwoStageFeatureSelector
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


ModuleNotFoundError: No module named 'FS'

In [2]:
df = pd.read_csv('F:/Kỳ 4/Thầy Hùng/OpenFE/data/Bot-IoT/bot_10k.csv')

C:\Users\chaut\AppData\Local\Temp\ipykernel_27656\2983648171.py:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('F:/Kỳ 4/Thầy Hùng/OpenFE/data/Bot-IoT/bot_10k.csv')


In [3]:
df.head(5)

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1933540,1.528096e+09,e s,2,tcp,1,192.168.100.149,32204,192.168.100.3,80,...,0.316896,100,100,0.311905,0.316896,480,480,1,DDoS,TCP
1,2274588,1.528096e+09,e s,2,tcp,1,192.168.100.148,2615,192.168.100.3,80,...,0.396835,100,100,0.527351,0.396835,70,70,1,DDoS,TCP
2,2520343,1.528096e+09,e s,2,tcp,1,192.168.100.147,13359,192.168.100.3,80,...,0.163101,100,100,0.107083,0.163101,176,176,1,DDoS,TCP
3,2559033,1.528096e+09,e s,2,tcp,1,192.168.100.148,14074,192.168.100.3,80,...,0.172727,100,94,0.208418,0.172727,672,636,1,DDoS,TCP
4,2769855,1.528099e+09,e,1,udp,3,192.168.100.147,62334,192.168.100.3,80,...,0.920289,100,93,0.937392,0.920289,1279,1209,1,DDoS,UDP


In [4]:
df.shape

(30556, 46)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30556 entries, 0 to 30555
Data columns (total 46 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   pkSeqID                           30556 non-null  int64  
 1   stime                             30556 non-null  float64
 2   flgs                              30556 non-null  object 
 3   flgs_number                       30556 non-null  int64  
 4   proto                             30556 non-null  object 
 5   proto_number                      30556 non-null  int64  
 6   saddr                             30556 non-null  object 
 7   sport                             30556 non-null  object 
 8   daddr                             30556 non-null  object 
 9   dport                             30556 non-null  object 
 10  pkts                              30556 non-null  int64  
 11  bytes                             30556 non-null  int64  
 12  stat

In [6]:
df['attack'].unique()

array([1, 0], dtype=int64)

In [7]:
df['category'].value_counts()

category
DDoS              10000
DoS               10000
Reconnaissance    10000
Normal              477
Theft                79
Name: count, dtype: int64

In [8]:
df.select_dtypes(include=['object']).columns

Index(['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state',
       'category', 'subcategory'],
      dtype='object')

In [9]:
columns_to_convert = ['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state', 'subcategory']

In [10]:
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [16]:
df.dropna()

,pkSeqID,stime,flgs_number,proto_number,sport,dport,pkts,bytes,state_number,ltime,...,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category
0,1933540,1.528096e+09,2,1,32204.0,80.0,5,770,3,1.528096e+09,...,0.316896,0.316896,100,100,0.311905,0.316896,480,480,1,0
1,2274588,1.528096e+09,2,1,2615.0,80.0,7,890,1,1.528096e+09,...,0.396835,0.396835,100,100,0.527351,0.396835,70,70,1,0
2,2520343,1.528096e+09,2,1,13359.0,80.0,4,616,3,1.528096e+09,...,0.163101,0.163101,100,100,0.107083,0.163101,176,176,1,0
3,2559033,1.528096e+09,2,1,14074.0,80.0,9,1010,1,1.528096e+09,...,0.174369,0.172727,100,94,0.208418,0.172727,672,636,1,0
4,2769855,1.528099e+09,1,3,62334.0,80.0,13,780,4,1.528099e+09,...,0.937450,0.920289,100,93,0.937392,0.920289,1279,1209,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30551,3668518,1.529381e+09,1,1,35064.0,22.0,6,434,1,1.529381e+09,...,9.889330,9.889330,19,19,455.754000,9.889330,30,30,1,4
30552,3668519,1.529381e+09,1,1,35066.0,22.0,6,434,1,1.529381e+09,...,9.889330,9.889330,19,19,10453.000000,9.889330,30,30,1,4
30553,3668520,1.529381e+09,1,1,35070.0,22.0,31,5472,6,1.529381e+09,...,9.889330,9.889330,19,19,10.785200,9.889330,441,441,1,4
30554,3668521,1.529381e+09,1,1,43001.0,4433.0,2,134,1,1.529381e+09,...,22346.400000,666667.000000,1,3,666667.000000,22346.400000,2,4,1,4


In [17]:
df.isnull().sum()

pkSeqID                               0
stime                                 0
flgs_number                           0
proto_number                          0
sport                               944
dport                               944
pkts                                  0
bytes                                 0
state_number                          0
ltime                                 0
seq                                   0
dur                                   0
mean                                  0
stddev                                0
sum                                   0
min                                   0
max                                   0
spkts                                 0
dpkts                                 0
sbytes                                0
dbytes                                0
rate                                  0
srate                                 0
drate                                 0
TnBPSrcIP                             0


In [18]:
#Encode
label_encoder = LabelEncoder()
cols_to_encode = ['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state', 'category', 'subcategory']
for col in cols_to_encode:
    if df[col].dtype == 'object':
        df[col] = label_encoder.fit_transform(df[col])

KeyError: 'flgs'

In [ ]:
for col in df.columns:
  if len(df[col].unique()) <= 1:
    print(col)
    df.drop(col, axis=1, inplace=True)

flgs
proto
saddr
daddr
state
subcategory


In [ ]:
features = df.drop(['attack'], axis=1)
labels = df[['category']]

In [ ]:
features.head()

,pkSeqID,stime,flgs_number,proto_number,sport,dport,pkts,bytes,state_number,ltime,...,TnP_Per_Dport,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,category
0,1933540,1.528096e+09,2,1,32204.0,80.0,5,770,3,1.528096e+09,...,508,0.316896,0.316896,100,100,0.311905,0.316896,480,480,DDoS
1,2274588,1.528096e+09,2,1,2615.0,80.0,7,890,1,1.528096e+09,...,520,0.396835,0.396835,100,100,0.527351,0.396835,70,70,DDoS
2,2520343,1.528096e+09,2,1,13359.0,80.0,4,616,3,1.528096e+09,...,612,0.163101,0.163101,100,100,0.107083,0.163101,176,176,DDoS
3,2559033,1.528096e+09,2,1,14074.0,80.0,9,1010,1,1.528096e+09,...,744,0.174369,0.172727,100,94,0.208418,0.172727,672,636,DDoS
4,2769855,1.528099e+09,1,3,62334.0,80.0,13,780,4,1.528099e+09,...,1279,0.937450,0.920289,100,93,0.937392,0.920289,1279,1209,DDoS


In [19]:
df['category']

0        0
1        0
2        0
3        0
4        0
        ..
30551    4
30552    4
30553    4
30554    4
30555    4
Name: category, Length: 30556, dtype: int32

## Training

In [20]:
# X, _, y, _ = train_test_split(features, labels, train_size=500000, stratify=labels, random_state=42)

In [21]:
# y.value_counts()

attack
1         499935
0             65
Name: count, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

In [21]:
y_train.value_counts()

category
1           7498
3           7497
0           7495
2            369
4             58
Name: count, dtype: int64

## Generate new feature by OpenFE

In [22]:
ofe = OpenFE()
features = ofe.fit(data = X_train, label = y_train) 

The number of candidate features is 6963
Start stage I selection.


100%|██████████| 4/4 [14:52<00:00, 223.17s/it]


1305 same features have been deleted.


100%|██████████| 4/4 [10:14<00:00, 153.75s/it]


Meet early-stopping in successive feature-wise halving.


100%|██████████| 4/4 [34:13<00:00, 513.46s/it]


The number of remaining candidate features is 2000
Start stage II selection.


100%|██████████| 4/4 [00:21<00:00,  5.29s/it]


Finish data processing.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.555493 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 478998
[LightGBM] [Info] Number of data points in the train set: 18333, number of used features: 2039
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

In [26]:
# Set hyperparameters for binary classification
params = {
    'objective': 'multiclass',  # Đặt mục tiêu là phân loại nhị phân
    'metric': 'multi_error',  # Đánh giá bằng binary log loss
    'boosting_type': 'gbdt',
    'num_class': 5,
    'seed' : 1
}

## Score function

In [27]:
def get_score(train_x, test_x, train_y, test_y):
    # Chia dữ liệu thành tập huấn luyện và tập validation
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
    
    # Khởi tạo và đào tạo mô hình RandomForestClassifier
    rf = RandomForestClassifier(random_state=42)
    rf.fit(train_x, train_y)
    
    # Dự đoán trên tập kiểm tra và tính toán độ chính xác
    pred = rf.predict(test_x)
    score = accuracy_score(test_y, pred)
    
    return score

### Before Feature Generation

In [28]:
# get baseline score
score = get_score(X_train, X_test, y_train, y_test)
print("The score before feature generation is", score)

The score before feature generation is 1.0


### After Feature Generation

In [29]:
X_train, X_test = transform(X_train, X_test, ofe.new_features_list[:10], n_jobs = 1) 

In [30]:
X_train.columns

Index(['pkSeqID', 'stime', 'flgs_number', 'proto_number', 'sport', 'dport',
       'pkts', 'bytes', 'state_number', 'ltime', 'seq', 'dur', 'mean',
       'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes',
       'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP',
       'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP',
       'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP',
       'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport',
       'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP',
       'category', 'autoFE_f_0', 'autoFE_f_1', 'autoFE_f_2', 'autoFE_f_3',
       'autoFE_f_4', 'autoFE_f_5', 'autoFE_f_6', 'autoFE_f_7', 'autoFE_f_8',
       'autoFE_f_9'],
      dtype='object')

In [31]:
score = get_score(X_train, X_test, y_train, y_test)

In [32]:
# OpenFE recommends a list of new features. We include the top 10
# generated features to see how they influence the model performance
print("The score after feature generation is", score)
print("The top 10 generated features are")
for feature in ofe.new_features_list[:10]:
    print(tree_to_formula(feature))

The score after feature generation is 1.0
The top 10 generated features are
freq(category)
GroupByThenMean(pkts,category)
(TnP_PDstIP/category)
GroupByThenMin(TnP_PerProto,category)
GroupByThenMean(dport,category)
GroupByThenMax(AR_P_Proto_P_DstIP,category)
min(spkts,category)
GroupByThenMedian(min,category)
GroupByThenMax(dpkts,category)
round(stime)
